# Generate synpuf disclosure risk

Compare synthetic PUFs trained from a 10% sample, both to the training set and a 10% holdout. Synthetic file (1) is from synthimpute random forests; (2) is from the synthpop R package.

## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
import synthimpute as si
import synpuf

/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: package 'RevoUtils' was built under R version 3.4.3 

  warnings.warn(x, RRuntimeWarning)


**UPDATE!**

In [2]:
SYNTHESIS_ID = 19
PCT_TRAIN = 100

Folders.

In [3]:
PUF_SAMPLE_DIR = '~/Downloads/puf/'
SYN_DIR = '~/Downloads/syntheses/'
NEAREST_DIR = '~/Downloads/nearest/'

### Load data

In [4]:
synth = pd.read_csv(SYN_DIR + 'synpuf' + str(SYNTHESIS_ID) + '.csv')
train = pd.read_csv(PUF_SAMPLE_DIR + 'puf.csv')

## Preprocessing

Drop calculated features used as seeds, and drop s006.

In [5]:
synpuf.add_subtracted_features(train)
DROPS = ['S006', 'e00600_minus_e00650', 'e01500_minus_e01700',
         'RECID', 'E00100', 'E09600']
train.drop(DROPS, axis=1, inplace=True)
synth.columns = [x.upper() for x in synth.columns]
synth = synth[train.columns]

In [6]:
synth.columns

Index(['DSI', 'EIC', 'FDED', 'F2441', 'F6251', 'MARS', 'MIDR', 'N24', 'XTOT',
       'E00200', 'E00300', 'E00400', 'E00650', 'E00700', 'E00800', 'E00900',
       'E01100', 'E01200', 'E01400', 'E01700', 'E02000', 'E02100', 'E02300',
       'E02400', 'E03150', 'E03210', 'E03220', 'E03230', 'E03270', 'E03240',
       'E03290', 'E03300', 'E03400', 'E03500', 'E07240', 'E07260', 'E07300',
       'E07400', 'E07600', 'P08000', 'E09700', 'E09800', 'E09900', 'E11200',
       'E17500', 'E18400', 'E18500', 'E19200', 'E19800', 'E20100', 'E20400',
       'P22250', 'P23250', 'E24515', 'E24518', 'E26270', 'E27200', 'E32800',
       'E58990', 'E62900', 'E87521', 'E87530', 'E00600', 'E01500'],
      dtype='object')

In [7]:
synth.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

## Nearest calculation

Compare nearest standardized Euclidean distance. Takes ~10 hours.

In [8]:
%%time
nearest = si.nearest_record(synth.sample(frac=0.01), train, k=3, scale=True)

CPU times: user 5min 9s, sys: 382 ms, total: 5min 9s
Wall time: 5min 19s


In [9]:
nearest.to_csv(NEAREST_DIR + 'nearest' + str(SYNTHESIS_ID) + '.csv',
               index=False)

In [10]:
n = nearest.copy(deep=True)

In [11]:
n['min_dist'] = n[['dist1', 'dist2', 'dist3']].min(axis=1)
n['max_dist'] = n[['dist1', 'dist2', 'dist3']].max(axis=1)
n['mid_dist'] = n[['dist1', 'dist2', 'dist3']].sum(axis=1) - n.min_dist - n.max_dist

In [12]:
n['min_id'] = np.where(n.min_dist == n.dist1, n.id_B1,
                       np.where(n.min_dist == n.dist2, n.id_B2, n.id_B3))
n['max_id'] = np.where(n.max_dist == n.dist1, n.id_B1,
                       np.where(n.max_dist == n.dist2, n.id_B2, n.id_B3))
n['mid_id'] = n[['id_B1', 'id_B2', 'id_B3']].sum(axis=1) - n.min_id - n.max_id

In [13]:
n['id_B1'] = n.min_id
n['id_B2'] = n.mid_id
n['id_B3'] = n.max_id

n['dist1'] = n.min_dist
n['dist2'] = n.mid_dist
n['dist3'] = n.max_dist

In [14]:
n[nearest.columns].describe()

,id_A,id_B1,dist1,id_B2,dist2,id_B3,dist3
count,8189.000000,8189.000000,8189.000000,8189.000000,8189.000000,8189.000000,8189.000000
mean,408247.440835,80032.443278,1.081254,78550.858102,1.227726,77808.400537,1.314152
std,236702.828076,45927.851420,2.615731,48342.867196,2.852204,46030.646529,3.020003
min,231.000000,15.000000,0.000000,-123039.000000,0.000000,22.000000,0.000000
25%,201078.000000,40697.000000,0.010564,39811.000000,0.019665,38394.000000,0.025777
50%,405459.000000,79977.000000,0.253818,78501.000000,0.334892,75710.000000,0.385035
75%,615916.000000,118502.000000,1.192497,117918.000000,1.392820,116358.000000,1.504100
max,818542.000000,163782.000000,74.086358,279208.000000,74.661010,163785.000000,75.261934


In [15]:
n[nearest.columns].to_csv(NEAREST_DIR + 'nearest' + str(SYNTHESIS_ID) + '.csv',
                          index=False)